# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [9]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [ ]:

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [8]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [10]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

## question 1
limit = 5
generator = square_root_generator(limit)
sum = 0
for sqrt_value in generator:
    sum = sum + sqrt_value

print(f"Sum : {sum}")


Sum : 8.382332347441762


In [11]:
## question 2
limit = 13
generator = square_root_generator(limit)
i = 1
for sqrt_value in generator:
    print(f"{i} : {sqrt_value}")
    i = i + 1

1 : 1.0
2 : 1.4142135623730951
3 : 1.7320508075688772
4 : 2.0
5 : 2.23606797749979
6 : 2.449489742783178
7 : 2.6457513110645907
8 : 2.8284271247461903
9 : 3.0
10 : 3.1622776601683795
11 : 3.3166247903554
12 : 3.4641016151377544
13 : 3.605551275463989


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX

In [26]:
import dlt
import duckdb

## question 3
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

pipeline = dlt.pipeline(pipeline_name="people_data",
                        destination='duckdb',
                        dataset_name='people_info')

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_1,
                    table_name="people",
                    write_disposition="replace")

print(info)
con = duckdb.connect("people_data.duckdb")
con.sql("SELECT * FROM people_info.people").show()
con.sql("SELECT sum(age) FROM people_info.people").show()

# merging 2nd generator
info2 = pipeline.run(people_2,
                    table_name="people",
                    write_disposition="append")
                    # write_disposition="merge",
                    # primary_key="id")
print(info2)
con.sql("SELECT * FROM people_info.people order by id").show()
con.sql("SELECT sum(age) FROM people_info.people").show() # 353


Pipeline people_data load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_info
The duckdb destination used duckdb:////content/people_data.duckdb location to store data
Load package 1708240400.2431142 is LOADED and contains no failed jobs
┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708240400.2431142 │ xP8VOjaAJkkqxQ │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708240400.2431142 │ QNk0dMkK6h/Kqg │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1708240400.2431142 │ BETb0abJeLOV4A │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1708240400.2431142 │ 9veIxHCnK/EF/

In [27]:
## question 4
pipeline = dlt.pipeline(pipeline_name="people_data",
                        destination='duckdb',
                        dataset_name='people_info')

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_1,
                    table_name="people",
                    write_disposition="replace")

print(info)
con = duckdb.connect("people_data.duckdb")
con.sql("SELECT * FROM people_info.people").show()
con.sql("SELECT sum(age) FROM people_info.people").show()

# merging 2nd generator
info2 = pipeline.run(people_2,
                    table_name="people",
                    write_disposition="merge",
                    primary_key="id")
print(info2)
con.sql("SELECT * FROM people_info.people order by id").show()
con.sql("SELECT sum(age) FROM people_info.people").show() # 266


Pipeline people_data load step completed in 0.19 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_info
The duckdb destination used duckdb:////content/people_data.duckdb location to store data
Load package 1708240449.45687 is LOADED and contains no failed jobs
┌───────┬──────────┬───────┬─────────┬──────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │   _dlt_load_id   │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │     varchar      │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼──────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708240449.45687 │ MKxxfJg8BjVlqg │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708240449.45687 │ 2epcbkV9ZNgj5A │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1708240449.45687 │ /f7IpYwGclVzIw │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1708240449.45687 │ dDTXOFbq1Zitpw │ NULL       │
│